In [ ]:
# Install necessary libraries if they are not present
!pip install requests
!pip install beautifulsoup4
!pip install pandas

In [ ]:
# Import relevant packages
import requests
from bs4 import BeautifulSoup
import pandas as pd
import datetime
import random

# Data Extraction through Web Scraping.

## Introduction.

Almost 10 years ago, the job of a data scientist was labeled by Harvard Business Review as "the sexiest job of the 21st century" [(Davenport & Patil 2012)](https://hbr.org/2012/10/data-scientist-the-sexiest-job-of-the-21st-century). Since then, there has been a steady increase in the demand for data experts, and it is expected that both job creation and salaries will continue to rise in the coming years.

The objective of this tutorial is to use web scraping techniques to extract data on job offers for data scientists published on an open job portal (www.linkedin.com/jobs).


#### Extract the list of job postings returned by your search on LinkedIn.

In [ ]:
# Define the position and location to scrape
position = 'data scientist'
url_friendly_position = position.replace(" ","%20")
location = 'Monterrey'
url_search = 'https://www.linkedin.com/jobs/search/?keywords=%s&location=%s'%(url_friendly_position, location)
print(url_search)

In [ ]:
# To prevent the website from thinking you are a bot, use one of the following headers when making the request:

# List of headers
headers = [
    {'User-Agent': 'Mozilla/5.0'},
    {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.84 Safari/537.36'},
    {'User-Agent': 'Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.0.0 Mobile Safari/537.36'},
    {'User-Agent': 'Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.0.0 Mobile Safari/537.36'},
    {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.84 Safari/537.36'}
]

# Randomly select one header
head = random.choice(headers)

# Print the selected header
print(f"Using header: {head}")


In [ ]:
# Obtain a list of jobs related to the position and location
response = requests.get(url_search, headers=head)
print(response)
soup = BeautifulSoup(response.text, 'html.parser')
joblist = soup.find('ul', class_="jobs-search__results-list")
alljobs = joblist.find_all('li')

#### Select only the first job posting from the list and extract the following information: job title, company name, location, and job URL.

In [ ]:
info = alljobs[0].find('div', class_="base-search-card__info")
title = info.find('h3', class_="base-search-card__title").text.strip()
company = info.find('h4', class_="base-search-card__subtitle").text.strip()
metadata = alljobs[0].find('div', class_="base-search-card__metadata")
location_element = metadata.find('span', class_="job-search-card__location")
location_job = location_element.text.strip()

joburl = alljobs[0].find('a', class_="base-card__full-link")['href']

# Information about the first job
print(f'Title: {title}')
print(f'Company: {company}')
print(f'Location: {location_job}')
print(f'URL: {joburl}')

In [ ]:
# Create a list of dictionaries with job information

jobs = []

for job in alljobs:
    info = job.find('div', class_="base-search-card__info")
    title = info.find('h3', class_="base-search-card__title").text.strip()
    company = info.find('h4', class_="base-search-card__subtitle").text.strip()
    
    metadata = job.find('div', class_="base-search-card__metadata")
    location_element = metadata.find('span', class_="job-search-card__location")
    location_job = location_element.text.strip()
    
    joburl = job.find('a', class_="base-card__full-link")['href']
    
    job_info = {
        'Location': location_job,
        'Title': title,
        'Company': company,
        'Url': joburl
    }

    jobs.append(job_info)

# Select the first job from the list and extract the relevant information
first_job = jobs[0]
location_job = first_job['Location']
title_job = first_job['Title']
company_job = first_job['Company']
joburl_job = first_job['Url']

# Print the information of the first job
print(f'Título: {title_job}')
print(f'Empresa: {company_job}')
print(f'Ubicación: {location_job}')
print(f'URL del trabajo: {joburl_job}')

#### Based on the previous points, write a routine to extract the information for location, job title, company name, and job URL for all the job postings returned by your LinkedIn search, and store the data in a pandas dataframe.


In [ ]:
df_jobs = pd.DataFrame(jobs, columns=['Location', 'Title', 'Company', 'Url'])
print(df_jobs.head(1))

#### Export your dataframe to a .csv file.

In [ ]:
# Exportar el DataFrame a un archivo CSV
date = datetime.datetime.now().strftime('%Y-%m-%d')
position = position.replace(" ", "_")
nombre_archivo = f'LinkedIn_{position}_{location}_{date}.csv'
df_jobs.to_csv(nombre_archivo, index=False)

### How many job postings does your dataframe contain, and how many results are there in total from the LinkedIn search?

In [ ]:
number_jobs = df_jobs['Url'].count()
all_jobs_linkedin = int(soup.find('span', {'class': 'results-context-header__job-count'}).text)
print(f'The number of job postings in the DataFrame is {number_jobs}, while the total number of results on LinkedIn is {all_jobs_linkedin}.')

If there is a discrepancy between these two results, it is likely because I have only extracted the job postings from the first page of LinkedIn results. LinkedIn displays a limited number of postings per page, and if there are more postings, they are shown on subsequent pages.

To extract all available results from LinkedIn, you would need to implement a loop that iterates through all the result pages. This would involve extracting the URL for the next page from the current page, then performing web scraping on that page and adding the information to the dataframe, and so on, until there are no more pages or results.